In [1]:
import os
import pandas as pd
from neo4j import GraphDatabase

### setup

In [2]:
db_api = "neo4j://77.234.216.102:7687"
db_login = "neo4j"
db_password = "bKJ2ONAjy1FdYuM"

In [3]:
driver = GraphDatabase.driver(
    db_api,
    auth=(db_login, db_password)
)
driver.verify_connectivity()

In [4]:
datasets = [name for name in os.listdir() if name not in ('.conda', '.DS_Store', '.idea')]
datasets

['dna_protein',
 'biogrid',
 'affimers',
 'aptamers',
 'Repeats_dataset',
 'Riboswitch_dataset',
 'Viral_dataset',
 'upload.ipynb',
 'binding_db',
 'Ribosomal_dataset',
 'antigen_nanobody',
 'antibodies',
 'miRNA_dataset']

### affimers

In [5]:
affimer_name = pd.read_csv("affimers/affimer_name.csv")
affimer_name

,name,type,subtype
0,SUMO-Affimer,Protein,Protein
1,Affimer-NP,Protein,Protein
2,K33-specific affimer,Protein,Protein
3,?,Protein,Protein
4,Affimer F4 with specificity for Fc gamma recep...,Protein,Protein
5,Affimer G3,Protein,Protein
6,Affimer K6,Protein,Protein
7,Affimer K3,Protein,Protein
8,Affimer K69,Protein,Protein
9,K6-specific affimer,Protein,Protein


In [6]:
affimer_name = affimer_name[affimer_name.name != '?']

In [7]:
affimer_name['type'] = affimer_name['type'].str.lower()
affimer_name['subtype'] = affimer_name['subtype'].str.lower()

/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2764868890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affimer_name['type'] = affimer_name['type'].str.lower()
/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2764868890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affimer_name['subtype'] = affimer_name['subtype'].str.lower()


In [8]:
affimer_name['subtype'] = 'affimer'

/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/1257514469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affimer_name['subtype'] = 'affimer'


In [9]:
affimer_name

,name,type,subtype
0,SUMO-Affimer,protein,affimer
1,Affimer-NP,protein,affimer
2,K33-specific affimer,protein,affimer
4,Affimer F4 with specificity for Fc gamma recep...,protein,affimer
5,Affimer G3,protein,affimer
6,Affimer K6,protein,affimer
7,Affimer K3,protein,affimer
8,Affimer K69,protein,affimer
9,K6-specific affimer,protein,affimer
10,Affimer 9,protein,affimer


In [12]:
affimer_name_dict = affimer_name.to_dict("records")

with driver.session(database="neo4j") as session:
    query = "MERGE (n:$type:$subtype {name: $name})"
    for item in affimer_name_dict:
        raw_query = "MERGE (n:{type}:{subtype}".format(type=item["type"], subtype=item["subtype"])
        raw_query += " {name: $name})"
        result = session.run(raw_query, name=item["name"])

In [18]:
protein_name_affimer = pd.read_csv('affimers/protein_name_affimer.csv')
protein_name_affimer

,name,type,subtype
0,VelcroVax tandem HBcAg,Protein,Protein
1,Nucleocapsid,Protein,Protein
2,K33 diUb,Protein,Protein
3,RNA2 polyprotein,Protein,Protein
4,Low affinity immunoglobulin gamma Fc region re...,Protein,Protein
5,Low affinity immunoglobulin gamma Fc region re...,Protein,Protein
6,GTPase KRas,Protein,Protein
7,GTPase KRas,Protein,Protein
8,GTPase KRas,Protein,Protein
9,K6 diUb,Protein,Protein


In [19]:
protein_name_affimer = protein_name_affimer.drop_duplicates()

In [20]:
protein_name_affimer["type"] = protein_name_affimer["type"].str.lower()
protein_name_affimer["subtype"] = protein_name_affimer["subtype"].str.lower()
protein_name_affimer

/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2883162292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_name_affimer["type"] = protein_name_affimer["type"].str.lower()
/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2883162292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_name_affimer["subtype"] = protein_name_affimer["subtype"].str.lower()


,name,type,subtype
0,VelcroVax tandem HBcAg,protein,protein
1,Nucleocapsid,protein,protein
2,K33 diUb,protein,protein
3,RNA2 polyprotein,protein,protein
4,Low affinity immunoglobulin gamma Fc region re...,protein,protein
6,GTPase KRas,protein,protein
9,K6 diUb,protein,protein
10,Alpha-actinin-2,protein,protein
11,Small ubiquitin-related modifier 2,protein,protein
13,Serine/threonine-protein kinase PAK 5,protein,protein


In [21]:
protein_name_affimer_dict = protein_name_affimer.to_dict("records")

with driver.session(database="neo4j") as session:
    for item in protein_name_affimer_dict:
        raw_query = "MERGE (n:{type}:{subtype}".format(type=item["type"], subtype=item["subtype"])
        raw_query += " {name: $name})"
        result = session.run(raw_query, name=item["name"])

In [22]:
affimer_binding = pd.read_csv("affimers/affimer_binding.csv")
affimer_binding

,name_protein,name_affimer,structure_id
0,VelcroVax tandem HBcAg,SUMO-Affimer,7ZQ8
1,Nucleocapsid,Affimer-NP,6Z0O
2,K33 diUb,K33-specific affimer,5OHV
3,RNA2 polyprotein,?,6QOZ
4,Low affinity immunoglobulin gamma Fc region re...,Affimer F4 with specificity for Fc gamma recep...,5ML9
5,Low affinity immunoglobulin gamma Fc region re...,Affimer G3,5MN2
6,GTPase KRas,Affimer K6,6YR8
7,GTPase KRas,Affimer K3,6YXW
8,GTPase KRas,Affimer K69,7NY8
9,K6 diUb,K6-specific affimer,5OHL


In [23]:
affimer_binding = affimer_binding[affimer_binding["name_affimer"] != "?"]
affimer_binding

,name_protein,name_affimer,structure_id
0,VelcroVax tandem HBcAg,SUMO-Affimer,7ZQ8
1,Nucleocapsid,Affimer-NP,6Z0O
2,K33 diUb,K33-specific affimer,5OHV
4,Low affinity immunoglobulin gamma Fc region re...,Affimer F4 with specificity for Fc gamma recep...,5ML9
5,Low affinity immunoglobulin gamma Fc region re...,Affimer G3,5MN2
6,GTPase KRas,Affimer K6,6YR8
7,GTPase KRas,Affimer K3,6YXW
8,GTPase KRas,Affimer K69,7NY8
9,K6 diUb,K6-specific affimer,5OHL
10,Alpha-actinin-2,Affimer 9,6SWT


In [28]:
affimer_binding_dict = affimer_binding.to_dict("records")

with driver.session(database="neo4j") as session:
    for item in affimer_binding_dict:
        raw_query = """
        MATCH (first:protein),(second:affimer) 
        WHERE first.name="{protein_name}" AND second.name="{affimer_name}" 
        CREATE (second)-[:interacts_with]->(first)
        """.format(protein_name=item["name_protein"], affimer_name=item["name_affimer"])
        result = session.run(raw_query)


        MATCH (first:protein),(second:affimer) 
        WHERE first.name="VelcroVax tandem HBcAg" AND second.name="SUMO-Affimer" 
        CREATE (second)-[:interacts_with]->(first)
        

        MATCH (first:protein),(second:affimer) 
        WHERE first.name="Nucleocapsid" AND second.name="Affimer-NP" 
        CREATE (second)-[:interacts_with]->(first)
        

        MATCH (first:protein),(second:affimer) 
        WHERE first.name="K33 diUb" AND second.name="K33-specific affimer" 
        CREATE (second)-[:interacts_with]->(first)
        

        MATCH (first:protein),(second:affimer) 
        WHERE first.name="Low affinity immunoglobulin gamma Fc region receptor III-A" AND second.name="Affimer F4 with specificity for Fc gamma receptor IIIa" 
        CREATE (second)-[:interacts_with]->(first)
        

        MATCH (first:protein),(second:affimer) 
        WHERE first.name="Low affinity immunoglobulin gamma Fc region receptor III-A" AND second.name="Affimer G3" 
        CRE

### antibodies

In [29]:
antibodies = pd.read_csv('antibodies/antibodies.csv')
antibodies

,Unnamed: 0,name,type,subtype,representation_type,content,annotation
0,0,9cq4_H_L,protein,antibody,sequence,QVQLQQSGAELARPGASVKMSCKASGYTFTRYTMHWVKQRPGQGLE...,['G115 gamma delta TCR/CD3 complex bound by OK...
1,1,9cq8_C_E,protein,antibody,sequence,QVQLQQSGAELVRPGASVTLSCKASGYTFTDYEVYWVKQTPVHGLE...,['Dimeric 9C2 gamma delta TCR extracellular do...
2,2,9cq8_G_H,protein,antibody,sequence,QVQLQQSGAELVRPGASVTLSCKASGYTFTDYEVYWVKQTPVHGLE...,['Dimeric 9C2 gamma delta TCR extracellular do...
3,3,9cql_J_K,protein,antibody,sequence,QVQLQQPGADLVRPGTSVKLSCKASGYTFTSYWMHWVQQRPGQGLE...,['Dimeric 9C2 gamma delta TCR bound by Fab 3']
4,4,9cql_C_D,protein,antibody,sequence,QVQLQQPGADLVRPGTSVKLSCKASGYTFTSYWMHWVQQRPGQGLE...,['Dimeric 9C2 gamma delta TCR bound by Fab 3']
...,...,...,...,...,...,...,...
11771,11771,1yek_H_L,protein,antibody,sequence,EMQLQQSGAELLRPGTSVKLSCKTSGYIFTSYWIHWVKQRSGQGLE...,['CATALYTIC ANTIBODY D2.3 COMPLEX']
11772,11772,4s1d_H_L,protein,antibody,sequence,EVQLQQSGAELVKPGASVKLSCTSSGFNNKDTFFQWVKQRPEEGLE...,['Structure of IgG1 Fab fragment in complex wi...
11773,11773,4yhi_H_L,protein,antibody,sequence,QVQLVQSGAEVKKPGASVKVSCKASGYTFTDYYMHWVRQAPGQGLE...,['Reversal Agent for Dabigatran']
11774,11774,1oar_I_M,protein,antibody,sequence,EVQLQQSGAELVKPGASVKLSCKASGYTFTSYWMHWVKQRPGRGLE...,['Fv IgE SPE-7 in complex with Alizarin Red']


In [30]:
antibodies.drop('Unnamed: 0', axis=1, inplace=True)

In [31]:
antibodies.drop_duplicates()

,name,type,subtype,representation_type,content,annotation
0,9cq4_H_L,protein,antibody,sequence,QVQLQQSGAELARPGASVKMSCKASGYTFTRYTMHWVKQRPGQGLE...,['G115 gamma delta TCR/CD3 complex bound by OK...
1,9cq8_C_E,protein,antibody,sequence,QVQLQQSGAELVRPGASVTLSCKASGYTFTDYEVYWVKQTPVHGLE...,['Dimeric 9C2 gamma delta TCR extracellular do...
2,9cq8_G_H,protein,antibody,sequence,QVQLQQSGAELVRPGASVTLSCKASGYTFTDYEVYWVKQTPVHGLE...,['Dimeric 9C2 gamma delta TCR extracellular do...
3,9cql_J_K,protein,antibody,sequence,QVQLQQPGADLVRPGTSVKLSCKASGYTFTSYWMHWVQQRPGQGLE...,['Dimeric 9C2 gamma delta TCR bound by Fab 3']
4,9cql_C_D,protein,antibody,sequence,QVQLQQPGADLVRPGTSVKLSCKASGYTFTSYWMHWVQQRPGQGLE...,['Dimeric 9C2 gamma delta TCR bound by Fab 3']
...,...,...,...,...,...,...
11771,1yek_H_L,protein,antibody,sequence,EMQLQQSGAELLRPGTSVKLSCKTSGYIFTSYWIHWVKQRSGQGLE...,['CATALYTIC ANTIBODY D2.3 COMPLEX']
11772,4s1d_H_L,protein,antibody,sequence,EVQLQQSGAELVKPGASVKLSCTSSGFNNKDTFFQWVKQRPEEGLE...,['Structure of IgG1 Fab fragment in complex wi...
11773,4yhi_H_L,protein,antibody,sequence,QVQLVQSGAEVKKPGASVKVSCKASGYTFTDYYMHWVRQAPGQGLE...,['Reversal Agent for Dabigatran']
11774,1oar_I_M,protein,antibody,sequence,EVQLQQSGAELVKPGASVKLSCKASGYTFTSYWMHWVKQRPGRGLE...,['Fv IgE SPE-7 in complex with Alizarin Red']


In [32]:
antibodies['type'] = antibodies['type'].str.lower()
antibodies['subtype'] = antibodies['subtype'].str.lower()

In [38]:
antibodies[antibodies['content'].isna()]

,name,type,subtype,representation_type,content,annotation
11578,2a9n_H_L,protein,antibody,sequence,NaN,['A Mutation Designed to Alter Crystal Packing...
11704,2a9n_I_M,protein,antibody,sequence,NaN,['A Mutation Designed to Alter Crystal Packing...


In [61]:
antibodies['subtype'].unique()

array(['antibody'], dtype=object)

In [40]:
antibodies.iloc[11578]

name                                                            2a9n_H_L
type                                                             protein
subtype                                                         antibody
representation_type                                             sequence
content                                                              NaN
annotation             ['A Mutation Designed to Alter Crystal Packing...
Name: 11578, dtype: object

In [43]:
antibodies_dict[11579:]

[{'name': '1dbk_H_L',
  'type': 'protein',
  'subtype': 'antibody',
  'representation_type': 'sequence',
  'content': 'QIQLVQSGPELKKPGETVKISCKASGYAFTNYGVNWVKEAPGKELKWMGWINIYTGEPTYVDDFKGRFAFSLETSASTAYLEINNLKNEDTATYFCTRGDYVNWYFDVWGAGTTVTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSPRPSETVTCNVAHPASSTKVDKKIVPR|ASPVALVALMETTHRGLNILEPROLEUSERLEUPROVALASNLEUGLYASPGLNALASERILESERCYSARGSERSERGLNSERLEUILEHISSERASNGLYASNTHRTYRLEUHISTRPTYRLEUGLNLYSPROGLYGLNSERPROLYSLEULEUMETTYRLYSVALSERASNARGPHETYRGLYVALPROASPARGPHESERGLYSERGLYSERGLYTHRASPPHETHRLEULYSILESERARGVALGLUALAGLUASPLEUGLYILETYRPHECYSSERGLNSERSERHISVALPROPROTHRPHEGLYGLYGLYTHRLYSLEUGLUILELYSARGALAASPALAALAPROTHRVALSERILEPHEPROPROSERSERGLUGLNLEUTHRSERGLYGLYALASERVALVALCYSPHELEUASNASNPHETYRPROLYSASPILEASNVALLYSTRPLYSILEASPGLYSERGLUARGGLNASNGLYVALLEUASNSERTRPTHRASPGLNASPSERLYSASPSERTHRTYRSERMETSERSERTHRLEUTHRLEUTHRLYSASPGLUTYRGLUARGHISASNSERTYRTHRCYSGLUALATHRHISLYSTHRSERTHRSERPROILEVALLYSSERPHEASNAR',
  'annotati

In [45]:
antibodies_dict = antibodies.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in antibodies_dict[11705:]:
        raw_query = "MERGE (n:{type}:{subtype}".format(type=item["type"], subtype=item["subtype"])
        raw_query += " {name: $name, representation_type: $representation_type, content: $content, annotation: $annotation})"
        result = session.run(
            raw_query,
            name=item["name"],
            representation_type=item["representation_type"],
            content=item["content"],
            annotation=item["annotation"],
        )

In [47]:
molecules_from_antibodies = pd.read_csv("antibodies/molecules_from_antibodies.csv", index_col='Unnamed: 0')
molecules_from_antibodies

,name,type,content,representation_type
0,cep1 peptide (from enolase),peptide,ACACKIHACIREIFDSCIRGNPTVECK,sequence
1,glucose-6-phosphate isomerase,peptide,ACACFEQLLSGAHWMDQHFCA,sequence
2,ring-infected erythrocyte surface antigen peptide,peptide,ACAEENVEENVEENVEENVGG,sequence
3,p/a#1 epitope peptide,peptide,ACEAPAPAAPA,sequence
4,hiv fusion peptide 512-519,peptide,ACEAVGLGAVF,sequence
...,...,...,...,...
3507,major prion protein,protein,VVGGLGGYMLGSAMSRPMIHFGNDWEDRYYRENMYRYPNQVYYRPV...,sequence
3508,coagulation factor xii,protein,VVGGLVALRGAHPYIAALYWGHSFCAGSLIAPCWVLTAAHCLQDRP...,sequence
3509,suppressor of tumorigenicity 14 protein,protein,VVGGTDADEGEWPWQVSLHALGQGHICGASLISPNWLVSAAHCYID...,sequence
3510,vascular endothelial growth factor,protein,VVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGG...,sequence


In [49]:
molecules_from_antibodies['type'] = molecules_from_antibodies['type'].str.lower()

In [51]:
molecules_from_antibodies.drop_duplicates(subset="name")

,name,type,content,representation_type
0,cep1 peptide (from enolase),peptide,ACACKIHACIREIFDSCIRGNPTVECK,sequence
1,glucose-6-phosphate isomerase,peptide,ACACFEQLLSGAHWMDQHFCA,sequence
2,ring-infected erythrocyte surface antigen peptide,peptide,ACAEENVEENVEENVEENVGG,sequence
3,p/a#1 epitope peptide,peptide,ACEAPAPAAPA,sequence
4,hiv fusion peptide 512-519,peptide,ACEAVGLGAVF,sequence
...,...,...,...,...
3505,protein (p-glycoprotein),peptide,VVQEALDKAREGRT,sequence
3506,coagulation factor ix heavy chain,protein,VVGGEDAKPGQFPWQVVLNGKVDAFCGGSIVNEKWIVTAAHCVETG...,sequence
3508,coagulation factor xii,protein,VVGGLVALRGAHPYIAALYWGHSFCAGSLIAPCWVLTAAHCLQDRP...,sequence
3509,suppressor of tumorigenicity 14 protein,protein,VVGGTDADEGEWPWQVSLHALGQGHICGASLISPNWLVSAAHCYID...,sequence


In [55]:
molecules_from_antibodies['type'] = molecules_from_antibodies['type'].str.replace(" ", "_")

In [56]:
molecules_from_antibodies_dict = molecules_from_antibodies.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in molecules_from_antibodies_dict[759:]:
        raw_query = "MERGE (n:{type}".format(type=item["type"])
        raw_query += " {name: $name, representation_type: $representation_type, content: $content})"
        result = session.run(
            raw_query,
            name=item["name"],
            representation_type=item["representation_type"],
            content=item["content"],
        )

In [79]:
antibodies_interaction = pd.read_csv("antibodies/antibodies_interactions.csv", index_col='Unnamed: 0')
antibodies_interaction

,antibody_name,ligand_name,kd
0,9cq4_H_L,t-cell surface glycoprotein cd3 epsilon chain,NaN
1,9cq8_C_E,9c2 tcr delta chain,NaN
2,9cq8_G_H,9c2 tcr delta chain,NaN
3,9cql_J_K,9c2 tcr delta chain,NaN
4,9cql_C_D,9c2 tcr delta chain,NaN
...,...,...,...
487,1yek_H_L,P-NITROPHENOL,NaN
488,4s1d_H_L,"N~6~-{5-[(3aS,4S,6aR)-2-oxohexahydro-1H-thieno...",NaN
489,4yhi_H_L,N-[(2-{[(4-carbamimidoylphenyl)amino]methyl}-1...,NaN
490,1oar_I_M,ALIZARIN RED,4.000000e-08


In [80]:
antibodies_interaction.drop_duplicates()

,antibody_name,ligand_name,kd
0,9cq4_H_L,t-cell surface glycoprotein cd3 epsilon chain,NaN
1,9cq8_C_E,9c2 tcr delta chain,NaN
2,9cq8_G_H,9c2 tcr delta chain,NaN
3,9cql_J_K,9c2 tcr delta chain,NaN
4,9cql_C_D,9c2 tcr delta chain,NaN
...,...,...,...
487,1yek_H_L,P-NITROPHENOL,NaN
488,4s1d_H_L,"N~6~-{5-[(3aS,4S,6aR)-2-oxohexahydro-1H-thieno...",NaN
489,4yhi_H_L,N-[(2-{[(4-carbamimidoylphenyl)amino]methyl}-1...,NaN
490,1oar_I_M,ALIZARIN RED,4.000000e-08


In [81]:
antibodies_interaction['ligand_type'] = [''] * antibodies_interaction.shape[0]

In [82]:
ligand_name_type = {
    item['name']: item['type'] for item in molecules_from_antibodies.to_dict("records")
}

In [83]:
antibodies_interaction['ligand_type'] = antibodies_interaction['ligand_name'].apply(lambda x: ligand_name_type.get(x))

In [84]:
antibodies_interaction = antibodies_interaction[~antibodies_interaction['ligand_type'].isna()]

In [85]:
antibodies_interaction

,antibody_name,ligand_name,kd,ligand_type
0,9cq4_H_L,t-cell surface glycoprotein cd3 epsilon chain,NaN,peptide
1,9cq8_C_E,9c2 tcr delta chain,NaN,protein
2,9cq8_G_H,9c2 tcr delta chain,NaN,protein
3,9cql_J_K,9c2 tcr delta chain,NaN,protein
4,9cql_C_D,9c2 tcr delta chain,NaN,protein
...,...,...,...,...
487,1yek_H_L,P-NITROPHENOL,NaN,small_molecule
488,4s1d_H_L,"N~6~-{5-[(3aS,4S,6aR)-2-oxohexahydro-1H-thieno...",NaN,small_molecule
489,4yhi_H_L,N-[(2-{[(4-carbamimidoylphenyl)amino]methyl}-1...,NaN,small_molecule
490,1oar_I_M,ALIZARIN RED,4.000000e-08,small_molecule


In [87]:
from tqdm import tqdm

In [88]:
antibodies_interaction_dict = antibodies_interaction.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(antibodies_interaction_dict):
        raw_query = f"""
        MATCH (first:antibody),(second:{item['ligand_type']})
        WHERE first.name="{item['antibody_name']}" AND second.name="{item['ligand_name']}"
        CREATE (first)-[:interacts_with]->(second)
        """
        result = session.run(
            raw_query,
        )

100%|██████████| 11272/11272 [24:16<00:00,  7.74it/s]


### antigen_nanobody

In [90]:
antigen_table_norepeat = pd.read_csv("antigen_nanobody/anntigens_table_norepeat.csv")
antigen_table_norepeat

,name,type,subtype
0,SARS-CoV-2 nucleocapsid (N),protein,antigen
1,HSA,protein,antigen
2,Trypsin,protein,antigen
3,Spike RBD,protein,antigen
4,Spike S2P,protein,antigen
...,...,...,...
60,carbonic anhydrase,Protein,antigen
61,C. difficile toxin A,Protein,antigen
62,human serum albumin (HSA),Protein,antigen
63,cholera toxin binding (SP-CTH8),Protein,antigen


In [95]:
antigen_table_norepeat['type'] = antigen_table_norepeat['type'].str.lower().apply(lambda x: x.strip())
antigen_table_norepeat['subtype'] = antigen_table_norepeat['subtype'].str.lower().apply(lambda x: x.strip())

In [96]:
antigen_table_norepeat['type'].unique()

array(['protein', 'glycoprotein', 'capside'], dtype=object)

In [97]:
antigen_table_norepeat_dict = antigen_table_norepeat.to_dict("records")
with driver.session(database='neo4j') as session:
    for item in antigen_table_norepeat_dict:
        raw_query = f"MERGE(n:{item['type']}:{item['subtype']} "
        raw_query += "{name: $name})"
        session.run(raw_query, name=item['name'])

In [98]:
nanobody_table = pd.read_csv("antigen_nanobody/nanobody_table.csv")
nanobody_table

,name,type,subtype,representation_type,content
0,NRL-N-C2,protein,nanobody,Sequence - Aho,EVQLQAS-GGGLVRPGGSLRLSCAASG-FTFSS-----YAMMWVR...
1,NRL-N-C2-hop,protein,nanobody,Sequence - Aho,EVQLQAS-GGGLVRPGGSLRLSCAASG-FTFSS-----YAMMWVR...
2,NRL-N-E2,protein,nanobody,Sequence - Aho,EVQLQAS-GGGLVQAGGSLRLSCAASG-RTDST-----QHMAWFR...
3,NRL-N-E2-hop,protein,nanobody,Sequence - Aho,EVQLQAS-GGGLVQAGGSLRLSCAASG-RTDST-----QHMAWFR...
4,NRL-N-E10,protein,nanobody,Sequence - Aho,DVQLQAS-GGGLVQAGGSLRLSCAASA-RTF-------YTMGWFR...
...,...,...,...,...,...
401,SP15-96-123StoR,protein,nanobody,Sequence - Aho,NaN
402,SP15-96-123KtoI,protein,nanobody,Sequence - Aho,NaN
403,SP15-96-123EKtoWI,protein,nanobody,Sequence - Aho,NaN
404,SP15-96-123EStoWR,protein,nanobody,Sequence - Aho,NaN


In [99]:
nanobody_table['type'] = nanobody_table['type'].str.lower().apply(lambda x: x.strip())
nanobody_table['subtype'] = nanobody_table['subtype'].str.lower().apply(lambda x: x.strip())

In [101]:
nanobody_table[nanobody_table['content'].isna()]

,name,type,subtype,representation_type,content
396,SP-CTH8,protein,nanobody,Sequence - Aho,NaN
397,Shark096,protein,nanobody,Sequence - Aho,NaN
398,SP15-096-123,protein,nanobody,Sequence - Aho,NaN
399,SP15-096-13,protein,nanobody,Sequence - Aho,NaN
400,SP15-96-123EtoW,protein,nanobody,Sequence - Aho,NaN
401,SP15-96-123StoR,protein,nanobody,Sequence - Aho,NaN
402,SP15-96-123KtoI,protein,nanobody,Sequence - Aho,NaN
403,SP15-96-123EKtoWI,protein,nanobody,Sequence - Aho,NaN
404,SP15-96-123EStoWR,protein,nanobody,Sequence - Aho,NaN
405,SP15-96-123SKtoRI,protein,nanobody,Sequence - Aho,NaN


In [102]:
nanobody_table_dict = nanobody_table.to_dict("records")
with driver.session(database='neo4j') as session:
    for item in nanobody_table_dict[396:]:
        raw_query = f"MERGE(n:{item['type']}:{item['subtype']} "
        raw_query += "{name: $name"
        kwargs = {"name": item['name']}
        # if item.get('content'):
        #     raw_query += ', content: $content'
        #     kwargs['content'] = item['content']
        raw_query += "})"
        session.run(raw_query, **kwargs)

In [103]:
interaction_table = pd.read_csv("antigen_nanobody/interection_table.csv")
interaction_table

,nanobody_name,antigens_name,Kd
0,NRL-N-C2,SARS-CoV-2 nucleocapsid (N),1.1
1,NRL-N-C2-hop,SARS-CoV-2 nucleocapsid (N),NaN
2,NRL-N-E2,SARS-CoV-2 nucleocapsid (N),0.8
3,NRL-N-E2-hop,SARS-CoV-2 nucleocapsid (N),NaN
4,NRL-N-E10,SARS-CoV-2 nucleocapsid (N),14
...,...,...,...
434,SP15-96-123StoR,Ebola virus’ Nucleo 2.3,NaN
435,SP15-96-123KtoI,Ebola virus’ Nucleo 2.2,NaN
436,SP15-96-123EKtoWI,Ebola virus’ Nucleo 3.6,NaN
437,SP15-96-123EStoWR,Ebola virus’ Nucleo 12,NaN


In [ ]:
interaction_table['nanobody_type'] = [''] * interaction_table.shape[0]
interaction_table['antigen_type'] = [''] * interaction_table.shape[0]

In [104]:
nanobody_types = {item['name']: item['subtype'] for item in nanobody_table.to_dict("records")}
antigen_types = {item['name']: item['subtype'] for item in antigen_table_norepeat.to_dict("records")}

In [107]:
antigen_types

{'SARS-CoV-2 nucleocapsid (N)': 'antigen',
 'HSA': 'antigen',
 'Trypsin': 'antigen',
 'Spike RBD': 'antigen',
 'Spike S2P': 'antigen',
 'sfGFP': 'antigen',
 'Acute Myeloid Leukemia Marker CD33': 'antigen',
 'lysozyme': 'antigen',
 'BcII β-lactamases': 'antigen',
 'TEM-1': 'antigen',
 'NmcA β-lactamase': 'antigen',
 'RR6': 'antigen',
 'human prostate-specific antigen': 'antigen',
 'variant surface glycoprotein': 'antigen',
 'green fluorescent protein': 'antigen',
 'Human Lysozyme': 'antigen',
 'HER2': 'antigen',
 'Botulinum neurotoxin A': 'antigen',
 'CEACAM6 (glycosylphosphoinositol (GPI)-linked cell surface PROTEIN)': 'antigen',
 'Abrin': 'antigen',
 'Xenopus nucleoporin 98kDa (Nup98)': 'antigen',
 'recombinant EGFR': 'antigen',
 'BclA': 'antigen',
 'gerQ': 'antigen',
 'SODA1': 'antigen',
 'SOD15': 'antigen',
 'p5303': 'antigen',
 'CD47': 'antigen',
 'Listeria surface antigen Internalin B': 'antigen',
 'mCherry receptor': 'antigen',
 'E2/E3E2 (Western equine encephalitis virus)': 'ant

In [108]:
interaction_table['nanobody_type'] = interaction_table['nanobody_name'].apply(lambda x: nanobody_types.get(x))
interaction_table['antigen_type'] = interaction_table['antigens_name'].apply(lambda x: antigen_types.get(x, 'antigen'))

In [111]:
interaction_table = interaction_table.dropna(subset='nanobody_type')

In [112]:
interaction_table

,nanobody_name,antigens_name,Kd,nanobody_type,antigen_type
0,NRL-N-C2,SARS-CoV-2 nucleocapsid (N),1.1,nanobody,antigen
1,NRL-N-C2-hop,SARS-CoV-2 nucleocapsid (N),NaN,nanobody,antigen
2,NRL-N-E2,SARS-CoV-2 nucleocapsid (N),0.8,nanobody,antigen
3,NRL-N-E2-hop,SARS-CoV-2 nucleocapsid (N),NaN,nanobody,antigen
4,NRL-N-E10,SARS-CoV-2 nucleocapsid (N),14,nanobody,antigen
...,...,...,...,...,...
434,SP15-96-123StoR,Ebola virus’ Nucleo 2.3,NaN,nanobody,antigen
435,SP15-96-123KtoI,Ebola virus’ Nucleo 2.2,NaN,nanobody,antigen
436,SP15-96-123EKtoWI,Ebola virus’ Nucleo 3.6,NaN,nanobody,antigen
437,SP15-96-123EStoWR,Ebola virus’ Nucleo 12,NaN,nanobody,antigen


In [123]:
import numpy as np

nan_kd = interaction_table[interaction_table['Kd'].isna()].replace(np.nan, None).to_dict('records')[0]
nan_kd

{'nanobody_name': 'NRL-N-C2-hop',
 'antigens_name': ' SARS-CoV-2 nucleocapsid (N)',
 'Kd': None,
 'nanobody_type': 'nanobody',
 'antigen_type': 'antigen'}

In [125]:
bool(nan_kd['Kd'])

False

In [126]:
interaction_table_dict = interaction_table.replace(np.nan, None).to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(interaction_table_dict):
        raw_query = f"""
        MATCH (first:{item['nanobody_type']}),(second:{item['antigen_type']})
        WHERE first.name="{item['nanobody_name']}" AND second.name="{item['antigens_name']}"
        """
        if item['Kd']:
            raw_query += "CREATE (first)-[:interacts_with {kd: $kd}]->(second)"
            session.run(raw_query, kd=item['Kd'])
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 406/406 [01:20<00:00,  5.04it/s]


#### aptamers

In [128]:
aptamers = pd.read_csv("aptamers/aptamers.csv", index_col='Unnamed: 0')
aptamers

,name,type,subtype,representation_type,content,URL
0,APIPred_1,RNA,aptamer,sequence,UCGGAGGUGGUUCAGCUCUGCAUCGACAGUUGGC,NaN
1,APIPred_2,DNA,aptamer,sequence,CGGGGTGTTGTCCTGTGCTCTCCGGAGAGCACAGGACAACACCCCG,NaN
2,APIPred_3,DNA,aptamer,sequence,GCTGCAGTTGCACTGAATTCGCCTCTCGCCTCCGTACACTTAGTCG...,NaN
3,APIPred_4,DNA,aptamer,sequence,TATTTGCCCTTGCAGGCCGCAGGAGTGCTAGCAGT,NaN
4,APIPred_5,RNA,aptamer,sequence,UCUCUGGGCUCUUAGGAGAACGGAUAGGAGUGUGCUCGCU,NaN
...,...,...,...,...,...,...
127,Aptagen_8070,DNA,aptamer,sequence,GCACACCGATGGCGGTCCTGTTTAGGGGGTGTGC,https://www.aptagen.com/aptamers/rd29c33-t7/
128,Aptagen_8387,DNA,aptamer,sequence,AGCTTGCTGCAGCGATTCTTGATCGCCACAGAGCT,https://www.aptagen.com/aptamers/ss24-pj-35-ap...
129,Aptagen_7975,DNA,aptamer,sequence,TTTTTTATCAGGGCTAAAGAGTGCAGAGTTACTTAG,https://www.aptagen.com/aptamers/lysozyme-apta...
130,Aptagen_7993,DNA,aptamer,sequence,CACCTGGGGGAGTATTGCGGAGGAAGGTTCCAGGTG,https://www.aptagen.com/aptamers/huizenga-atp-...


In [130]:
aptamers = aptamers.drop_duplicates()

In [134]:
import numpy as np


def prepare_dataset(ds: pd.DataFrame):
    ds = ds.drop_duplicates()
    if 'type' in ds.columns:
        ds['type'] = ds['type'].str.lower().str.strip()

    if 'subtype' in ds.columns:
        ds['subtype'] = ds['subtype'].str.lower().str.strip()
    
    ds.replace(np.nan, None, inplace=True)
    return ds


In [135]:
prepared_aptamers = prepare_dataset(aptamers)
prepared_aptamers

,name,type,subtype,representation_type,content,URL
0,APIPred_1,rna,aptamer,sequence,UCGGAGGUGGUUCAGCUCUGCAUCGACAGUUGGC,None
1,APIPred_2,dna,aptamer,sequence,CGGGGTGTTGTCCTGTGCTCTCCGGAGAGCACAGGACAACACCCCG,None
2,APIPred_3,dna,aptamer,sequence,GCTGCAGTTGCACTGAATTCGCCTCTCGCCTCCGTACACTTAGTCG...,None
3,APIPred_4,dna,aptamer,sequence,TATTTGCCCTTGCAGGCCGCAGGAGTGCTAGCAGT,None
4,APIPred_5,rna,aptamer,sequence,UCUCUGGGCUCUUAGGAGAACGGAUAGGAGUGUGCUCGCU,None
...,...,...,...,...,...,...
17,Aptagen_8070,dna,aptamer,sequence,GCACACCGATGGCGGTCCTGTTTAGGGGGTGTGC,https://www.aptagen.com/aptamers/rd29c33-t7/
18,Aptagen_8387,dna,aptamer,sequence,AGCTTGCTGCAGCGATTCTTGATCGCCACAGAGCT,https://www.aptagen.com/aptamers/ss24-pj-35-ap...
19,Aptagen_7975,dna,aptamer,sequence,TTTTTTATCAGGGCTAAAGAGTGCAGAGTTACTTAG,https://www.aptagen.com/aptamers/lysozyme-apta...
20,Aptagen_7993,dna,aptamer,sequence,CACCTGGGGGAGTATTGCGGAGGAAGGTTCCAGGTG,https://www.aptagen.com/aptamers/huizenga-atp-...


In [136]:
prepared_aptamers.to_csv('aptamers/prepared/aptamers.csv', index=False)

In [137]:
molecules_from_aptamers = pd.read_csv('aptamers/molecules_from_aptamers.csv', index_col='Unnamed: 0')
molecules_from_aptamers

,content,name,type,representation_type
0,MDSTNYISKLFEYAQRQGQISDIKFEEVGTDGPDHLKTFTLRVVIK...,dsRNA_activated_protein_kinase,protein,sequence
1,MDELFPLIFPAEPAQASGPYVEIIEQPKQRGMRFRYKCEGRSAGSI...,Human_Nuclear_Factor_kB_RelA_p65,protein,sequence
2,MDEVGAQVAAPMFIHQSLGRKRDLYYPMSNRLVQSQPQRRDEWNSK...,atSPL14,protein,sequence
3,PISPIDTVPVKLKPGMDGPKVKQWPLTEEKIKALTEICNEMEKEGK...,HIV-1-RT,protein,sequence
4,LQENFLPQPRQQHHGTLVLHYRPHREEAGMQHPCLWPGSSHCSDDS...,prothrombin,protein,sequence
...,...,...,...,...
4516,Nc1nc(N)nc(N)n1,Melamine,Small_molecule,smiles
4517,COP(N)(=S)Oc1ccccc1C(=O)OC(C)C,Isocarbophos,Small_molecule,smiles
4518,CCC(C)C(NC(=O)CNC(=O)C(CCC(N)=O)NC(=O)C1CCCN1C...,Lysozyme,Small_molecule,smiles
4519,Nc1ncnc2c1ncn2C1OC(COP(=O)(O)OP(=O)(O)OP(=O)(O...,ATP,Small_molecule,smiles


In [138]:
molecules_from_aptamers = prepare_dataset(molecules_from_aptamers)

/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2289267440.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['type'] = ds['type'].str.lower().str.strip()
/var/folders/w0/7pl8bw_97tv676fpb5jcz6kh0000gn/T/ipykernel_3394/2289267440.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds.replace(np.nan, None, inplace=True)


In [139]:
molecules_from_aptamers

,content,name,type,representation_type
0,MDSTNYISKLFEYAQRQGQISDIKFEEVGTDGPDHLKTFTLRVVIK...,dsRNA_activated_protein_kinase,protein,sequence
1,MDELFPLIFPAEPAQASGPYVEIIEQPKQRGMRFRYKCEGRSAGSI...,Human_Nuclear_Factor_kB_RelA_p65,protein,sequence
2,MDEVGAQVAAPMFIHQSLGRKRDLYYPMSNRLVQSQPQRRDEWNSK...,atSPL14,protein,sequence
3,PISPIDTVPVKLKPGMDGPKVKQWPLTEEKIKALTEICNEMEKEGK...,HIV-1-RT,protein,sequence
4,LQENFLPQPRQQHHGTLVLHYRPHREEAGMQHPCLWPGSSHCSDDS...,prothrombin,protein,sequence
...,...,...,...,...
4405,O=C1OC(C(O)CO)C(O)=C1O,Vitamin C,small_molecule,smiles
4406,Nc1nc(N)nc(N)n1,Melamine,small_molecule,smiles
4407,COP(N)(=S)Oc1ccccc1C(=O)OC(C)C,Isocarbophos,small_molecule,smiles
4408,CCC(C)C(NC(=O)CNC(=O)C(CCC(N)=O)NC(=O)C1CCCN1C...,Lysozyme,small_molecule,smiles


In [140]:
molecules_from_aptamers.to_csv('aptamers/prepared/molecules_from_aptamers.csv', index=False)

In [141]:
aptamer_interactions = pd.read_csv('aptamers/aptamers_interactions.csv', index_col="Unnamed: 0")
aptamer_interactions

,aptamer_name,target_name,pKd,measurement,affinity
0,APIPred_1,dsRNA_activated_protein_kinase,NaN,NaN,NaN
1,APIPred_2,Human_Nuclear_Factor_kB_RelA_p65,NaN,NaN,NaN
2,APIPred_3,atSPL14,NaN,NaN,NaN
3,APIPred_4,HIV-1-RT,NaN,NaN,NaN
4,APIPred_5,prothrombin,NaN,NaN,NaN
...,...,...,...,...,...
127,Aptagen_8070,Melamine,0.92-0.98 nM (reported value),NaN,NaN
128,Aptagen_8387,Isocarbophos,NaN,NaN,NaN
129,Aptagen_7975,Lysozyme,N/A µM (reported value),NaN,NaN
130,Aptagen_7993,ATP,956 µM (reported value),NaN,NaN


In [142]:
aptamer_interactions.replace(np.nan, None)

,aptamer_name,target_name,pKd,measurement,affinity
0,APIPred_1,dsRNA_activated_protein_kinase,None,None,None
1,APIPred_2,Human_Nuclear_Factor_kB_RelA_p65,None,None,None
2,APIPred_3,atSPL14,None,None,None
3,APIPred_4,HIV-1-RT,None,None,None
4,APIPred_5,prothrombin,None,None,None
...,...,...,...,...,...
127,Aptagen_8070,Melamine,0.92-0.98 nM (reported value),None,None
128,Aptagen_8387,Isocarbophos,None,None,None
129,Aptagen_7975,Lysozyme,N/A µM (reported value),None,None
130,Aptagen_7993,ATP,956 µM (reported value),None,None


In [144]:
aptamer_types = {item['name']: item['subtype'] for item in prepared_aptamers.to_dict("records")}
aptamer_molecule_types = {item['name']: item['type'] for item in molecules_from_aptamers.to_dict("records")}

In [148]:
aptamer_interactions['target_type'] = [''] * aptamer_interactions.shape[0]

In [151]:
aptamer_interactions['target_type'] = aptamer_interactions['target_name'].apply(lambda x: aptamer_molecule_types.get(x))
aptamer_interactions.isna().sum()

aptamer_name       0
target_name       28
pKd             3060
measurement     4386
affinity        4386
target_type       28
dtype: int64

In [152]:
aptamer_interactions['target_type'].unique()

array(['protein', 'small_molecule', None], dtype=object)

In [155]:
aptamer_interactions = aptamer_interactions.dropna(subset='target_type')

In [156]:
aptamer_interactions.to_csv('aptamers/prepared/aptamers_interaction.csv', index=False)

In [157]:
prepared_aptamers

,name,type,subtype,representation_type,content,URL
0,APIPred_1,rna,aptamer,sequence,UCGGAGGUGGUUCAGCUCUGCAUCGACAGUUGGC,None
1,APIPred_2,dna,aptamer,sequence,CGGGGTGTTGTCCTGTGCTCTCCGGAGAGCACAGGACAACACCCCG,None
2,APIPred_3,dna,aptamer,sequence,GCTGCAGTTGCACTGAATTCGCCTCTCGCCTCCGTACACTTAGTCG...,None
3,APIPred_4,dna,aptamer,sequence,TATTTGCCCTTGCAGGCCGCAGGAGTGCTAGCAGT,None
4,APIPred_5,rna,aptamer,sequence,UCUCUGGGCUCUUAGGAGAACGGAUAGGAGUGUGCUCGCU,None
...,...,...,...,...,...,...
17,Aptagen_8070,dna,aptamer,sequence,GCACACCGATGGCGGTCCTGTTTAGGGGGTGTGC,https://www.aptagen.com/aptamers/rd29c33-t7/
18,Aptagen_8387,dna,aptamer,sequence,AGCTTGCTGCAGCGATTCTTGATCGCCACAGAGCT,https://www.aptagen.com/aptamers/ss24-pj-35-ap...
19,Aptagen_7975,dna,aptamer,sequence,TTTTTTATCAGGGCTAAAGAGTGCAGAGTTACTTAG,https://www.aptagen.com/aptamers/lysozyme-apta...
20,Aptagen_7993,dna,aptamer,sequence,CACCTGGGGGAGTATTGCGGAGGAAGGTTCCAGGTG,https://www.aptagen.com/aptamers/huizenga-atp-...


In [160]:
prepared_aptamers[prepared_aptamers['content'].isna()]

,name,type,subtype,representation_type,content,URL
10,RSAPred_,rna,aptamer,sequence,None,None


In [165]:
prepared_aptamers_dict = prepared_aptamers.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(prepared_aptamers_dict[2899 + 1 :]):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += " {name: $name, representation_type: $representation_type, content: $content})"
        session.run(
            query,
            name=item["name"],
            representation_type=item["representation_type"],
            content=item["content"],
        )

100%|██████████| 1132/1132 [02:43<00:00,  6.94it/s]


In [167]:
molecules_from_aptamers_dict = molecules_from_aptamers.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(molecules_from_aptamers_dict[167 + 1:]):
        query = f"MERGE (n:{item['type']} "
        query += " {name: $name, representation_type: $representation_type, content: $content})"
        session.run(
            query,
            name=item["name"],
            representation_type=item["representation_type"],
            content=item["content"],
        )

100%|██████████| 456/456 [01:05<00:00,  7.01it/s]


In [171]:
aptamer_interactions.iloc[2891]

aptamer_name      RSAPred_39.0
target_name      Cibacron blue
pKd                          4
measurement                NaN
affinity                   NaN
target_type     small_molecule
Name: 0, dtype: object

In [178]:
aptamer_interactions_dict = aptamer_interactions.replace(np.nan, None).to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(aptamer_interactions_dict[2891 + 21:]):
        raw_query = f"""
        MATCH (first:aptamer),(second:{item['target_type']})
        WHERE first.name="{item['aptamer_name']}" AND second.name="{item['target_name']}"
        """
        kwargs = {}
        args = []
        if item['pKd']:
            args.append('pKd: $pKd')
            kwargs['pKd'] = item['pKd']

        if item['measurement']:
            args.append('measurement: $measurement')
            kwargs['measurement'] = item['measurement']

        if item['affinity']:
            args.append('affinity: $affinity')
            kwargs['affinity'] = item['affinity']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 1581/1581 [05:27<00:00,  4.82it/s]


## prepared

In [179]:
import os

In [ ]:
def upload_table(ds: pd.DataFrame):
    ds.replace(np.nan, None, inplace=True)
    ds_dict = ds.to_dict("records")
    with driver.session(database="neo4j") as session:
        for item in ds_dict:
            

### dna protein

In [180]:
protein_table = pd.read_csv('data/dna_protein_prepared/protein_table.csv')
protein_table

,Unnamed: 0,protein_name,type,subtype,representation_type,sequence
0,0,Interferon-activable protein 204,protein,protein,sequence,GAVLHSEPLTVMVLTATDPFEYESPEHEVKNMLHATVATVSQYFHV...
1,1,DNA-binding protein 7d,protein,protein,sequence,VKVKFKYKGEEKEVDTSKIKKVWRVGKFVSFTYDDNGKTGRGAVSE...
2,2,Histone H3.1,protein,protein,sequence,RYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLRFQ...
3,3,Histone H2B type 1-K,protein,protein,sequence,RYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLRFQ...
4,4,Histone H2A type 1-B/E,protein,protein,sequence,RYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLRFQ...
...,...,...,...,...,...,...
8413,10808,Histone H2B 1.1,protein,protein,sequence,KPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDL...
8414,10809,Histone H2A type 1,protein,protein,sequence,KPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDL...
8415,10810,Histone H4,protein,protein,sequence,KPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDL...
8416,10811,Histone H3.2,protein,protein,sequence,KPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDL...


In [184]:
protein_table.dropna(subset='protein_name', inplace=True)

In [185]:
ds_dict = protein_table.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['protein_name'],
            representation_type=item['representation_type'],
            content=item['sequence'],
        )

100%|██████████| 8413/8413 [15:56<00:00,  8.80it/s]


In [186]:
dna_table = pd.read_csv('data/dna_protein_prepared/dna_table.csv')
dna_table

,structure_id,type,subtype,representation_type,sequence
0,5z7d,dna,dna,sequence,CCATCAGAAAGAGAGCGCTCTCTTTCTGATGGCCATCAGAAAGAGA...
1,1wtq,dna,dna,sequence,GTAATTACGTAATTAC
2,6l9h,dna,dna,sequence,ATCTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGT...
3,1ng9,dna,dna,sequence,AGCTGCCAGGCACCAGTGACTGGTGCTTGGCAGCT
4,1zqj,dna,dna,sequence,CATTAGATCTAATG
...,...,...,...,...,...
7093,8vlr,dna,dna,sequence,TCTCTGCCTGTTCTTCCAAAAGTGTATTTAGAAACTGCTCCAACAA...
7094,9e1r,dna,dna,sequence,GCACAGGATGTATATATCTGACACGTGCCTGGAGACTAGGGAGTAA...
7095,9e1u,dna,dna,sequence,CACAGGATGTATATATCTGACACGTGCCTGGAGACTAGGGAGTAAT...
7096,9e1v,dna,dna,sequence,CACAGGATGTATATATCTGACACGTGCCTGGAGACTAGGGAGTAAT...


In [187]:
dna_table.replace(np.nan, None)
ds_dict = dna_table.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['structure_id'],
            representation_type=item['representation_type'],
            content=item['sequence'],
        )

100%|██████████| 7098/7098 [11:31<00:00, 10.26it/s]


In [188]:
dna_protein_interaction = pd.read_csv("data/dna_protein_prepared/dna_protein_interaction.csv")

In [189]:
ds_dict = dna_protein_interaction.replace(np.nan, None).to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:dna),(second:protein)
        WHERE first.name="{item['structure_id']}" AND second.name="{item['protein_name']}"
        """
        kwargs = {}
        args = []
        if item['binding_sites']:
            args.append('binding_sites: $binding_sites')
            kwargs['binding_sites'] = item['binding_sites']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 10768/10768 [23:08<00:00,  7.76it/s]


### mirna

In [190]:
mirna = pd.read_csv('data/mirna_prepared/mirna_rna.csv')

In [191]:
ds_dict = mirna.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 44/44 [00:04<00:00, 10.41it/s]


In [192]:
mirna_target = pd.read_csv('data/mirna_prepared/mirna_target.csv')

In [193]:
ds_dict = mirna_target.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 94/94 [00:09<00:00, 10.24it/s]


In [194]:
mirna_interaction = pd.read_csv('data/mirna_prepared/mirna_interaction.csv')

In [195]:
ds_dict = mirna_interaction.replace(np.nan, None).to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:mirna),(second:rna_inhibitor)
        WHERE first.name="{item['RNA_name']}" AND second.name="{item['small_molecule_name']}"
        """
        kwargs = {}
        args = []
        if item['kd']:
            args.append('kd: $kd')
            kwargs['kd'] = item['kd']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 146/146 [00:14<00:00,  9.85it/s]


### repeats

In [197]:
repeats_rna = pd.read_csv('data/repeats_prepared/repeats_rna.csv')

In [198]:
ds_dict = repeats_rna.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 48/48 [00:04<00:00, 10.72it/s]


In [199]:
repeats_target = pd.read_csv('data/repeats_prepared/repeats_target.csv')

In [201]:
repeats_target['subtype'] = repeats_target['subtype'].str.replace('/', '_')

In [202]:
ds_dict = repeats_target.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 44/44 [00:04<00:00, 10.60it/s]


In [207]:
repeats_interaction = pd.read_csv('data/repeats_prepared/repeats_interaction.csv')

In [208]:
ds_dict = repeats_interaction.replace(np.nan, None).to_dict("records")
first_type = repeats_rna['subtype'].unique()[0]
second_type = repeats_target['subtype'].unique()[0]
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:{first_type}),(second:{second_type})
        WHERE first.name="{item['RNA_name']}" AND second.name="{item['small_molecule_name']}"
        """
        kwargs = {}
        args = []
        if item['kd']:
            args.append('kd: $kd')
            kwargs['kd'] = item['kd']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 97/97 [00:09<00:00,  9.93it/s]


### ribosomal

In [209]:
ribosomal_rna = pd.read_csv('data/ribosomal_prepared/ribosomal_rna.csv')

In [210]:
ds_dict = ribosomal_rna.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 34/34 [00:03<00:00, 10.61it/s]


In [211]:
ribosomal_target = pd.read_csv('data/ribosomal_prepared/ribosomal_target.csv')

In [212]:
ds_dict = ribosomal_target.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 284/284 [00:26<00:00, 10.80it/s]


In [213]:
ribosomal_interaction = pd.read_csv('data/ribosomal_prepared/ribosomal_interaction.csv')

In [214]:
ds_dict = ribosomal_interaction.replace(np.nan, None).to_dict("records")
first_type = ribosomal_rna['subtype'].unique()[0]
second_type = ribosomal_target['subtype'].unique()[0]
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:{first_type}),(second:{second_type})
        WHERE first.name="{item['RNA_name']}" AND second.name="{item['small_molecule_name']}"
        """
        kwargs = {}
        args = []
        if item['kd']:
            args.append('kd: $kd')
            kwargs['kd'] = item['kd']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 295/295 [00:30<00:00,  9.74it/s]


### riboswitch

In [215]:
riboswitch_rna = pd.read_csv('data/riboswitch_prepared/riboswitch_rna.csv')

In [216]:
ds_dict = riboswitch_rna.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']}:{item['subtype']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 38/38 [00:04<00:00,  9.08it/s]


In [217]:
riboswitch_target = pd.read_csv('data/riboswitch_prepared/riboswitch_target.csv')

In [218]:
ds_dict = riboswitch_target.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 67/67 [00:06<00:00,  9.78it/s]


In [219]:
riboswitch_interaction = pd.read_csv('data/riboswitch_prepared/riboswitch_interaction.csv')

In [220]:
ds_dict = riboswitch_interaction.replace(np.nan, None).to_dict("records")
first_type = riboswitch_rna['subtype'].unique()[0]
second_type = riboswitch_target['type'].unique()[0]
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:{first_type}),(second:{second_type})
        WHERE first.name="{item['RNA_name']}" AND second.name="{item['small_molecule_name']}"
        """
        kwargs = {}
        args = []
        if item['kd']:
            args.append('kd: $kd')
            kwargs['kd'] = item['kd']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 100/100 [00:10<00:00,  9.75it/s]


### viral

In [221]:
viral_rna = pd.read_csv('data/viral_prepared/viral_rna.csv')

In [222]:
ds_dict = viral_rna.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 63/63 [00:06<00:00, 10.45it/s]


In [223]:
viral_target = pd.read_csv('data/viral_prepared/viral_target.csv')

In [224]:
ds_dict = viral_target.to_dict("records")
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        query = f"MERGE (n:{item['type']} "
        query += '{name: $name, representation_type: $representation_type, content: $content})'
        session.run(
            query,
            name=item['name'],
            representation_type=item['representation_type'],
            content=item['content'],
        )

100%|██████████| 204/204 [00:19<00:00, 10.49it/s]


In [225]:
viral_interaction = pd.read_csv('data/viral_prepared/viral_interaction.csv')

In [226]:
ds_dict = viral_interaction.replace(np.nan, None).to_dict("records")
first_type = viral_rna['subtype'].unique()[0]
second_type = viral_target['subtype'].unique()[0]
with driver.session(database="neo4j") as session:
    for item in tqdm(ds_dict):
        raw_query = f"""
        MATCH (first:{first_type}),(second:{second_type})
        WHERE first.name="{item['RNA_name']}" AND second.name="{item['small_molecule_name']}"
        """
        kwargs = {}
        args = []
        if item['kd']:
            args.append('kd: $kd')
            kwargs['kd'] = item['kd']

        if args:
            args_string = '{' + ', '.join(args) + '}'
            raw_query += f"CREATE (first)-[:interacts_with {args_string}]->(second)"
            session.run(raw_query, **kwargs)
        else:
            raw_query += "CREATE (first)-[:interacts_with]->(second)"
            session.run(raw_query)

100%|██████████| 281/281 [00:28<00:00,  9.91it/s]


### 